# Predicción y refinamiento de proteínas transmembrana con AlphaFold2 + RosettaMP
Este notebook permite:
1. Subir una secuencia FASTA.
2. Definir o predecir región transmembrana.
3. Predecir estructura con ColabFold (AlphaFold2).
4. Generar archivo `.span` para RosettaMP.
5. Compilar Rosetta.
6. Refinar estructura.
7. Visualizar el resultado.

In [ ]:
#@title Instalar dependencias
!pip install -q colabfold biopython py3Dmol requests


In [ ]:
#@title Subir archivo FASTA
from google.colab import files
uploaded = files.upload()
fasta_file = list(uploaded.keys())[0]


In [ ]:
#@title Ingresar región transmembrana (opcional)
import ipywidgets as widgets
from IPython.display import display

tm_input = widgets.Text(
    value='',
    placeholder='Ej: 45-67',
    description='TM región:',
    disabled=False
)
display(tm_input)


In [ ]:
#@title Predecir región TM si no se ingresó
import requests
from Bio import SeqIO

record = SeqIO.read(fasta_file, "fasta")
sequence = str(record.seq)

if tm_input.value.strip():
  tm_range = tm_input.value.strip()
else:
  print("Usando predicción con DeepTMHMM...")
  res = requests.post("https://deeptmhmmapi.biolib.com/predict", json={"sequence": sequence})
  tm_range = res.json().get("regions", ["40-60"])[0]  # Fallback

print("Región TM usada:", tm_range)


In [ ]:
#@title Ejecutar ColabFold
from colabfold.batch import run
from pathlib import Path
import os

out_dir = Path("prediction")
out_dir.mkdir(exist_ok=True)
res = run([("query", sequence)], result_dir=str(out_dir), use_templates=False, num_models=1)
print("Modelo guardado:", res["query"]["model"])

ranked_model_path = out_dir / "query_ranked_0.pdb"


In [ ]:
#@title Generar archivo .span para RosettaMP
start, end = map(int, tm_range.split('-'))

with open("protein.span", "w") as f:
    f.write("TM region prediction\n1 1\n{} {}".format(start, end))
print("Archivo span creado.")


In [ ]:
#@title Descargar y compilar RosettaMP (tarda ~30-45min)
!wget -q https://www.rosettacommons.org/downloads/academic/rosetta.latest.tar.bz2 -O rosetta.tar.bz2
!tar -xjf rosetta.tar.bz2
!cd rosetta*/main && ./scons.py -j2 mode=release bin/mp_relax


In [ ]:
#@title Ejecutar refinamiento con RosettaMP
import glob
import shutil

ranked_pdb = str(ranked_model_path)
shutil.copy(ranked_pdb, "ranked_0.pdb")

!rosetta*/main/source/bin/mp_relax.default.linuxgccrelease \
  -in:file:s ranked_0.pdb \
  -mp:setup:spanfiles protein.span \
  -relax:fast \
  -nstruct 1 \
  -out:path:all output/


In [ ]:
#@title Visualizar modelo original vs refinado
import py3Dmol

view = py3Dmol.view(width=600, height=400)
with open("ranked_0.pdb") as f: view.addModel(f.read(), "pdb")
with open("output/ranked_0_0001.pdb") as f: view.addModel(f.read(), "pdb")
view.setStyle({'model':0}, {'cartoon': {'color': 'blue'}})
view.setStyle({'model':1}, {'cartoon': {'color': 'red'}})
view.zoomTo()
view
